![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BertForQuestionAnswering.ipynb)

## Import ONNX BertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `BertForQuestionAnswering` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for question answering via `BertForQuestionAnswering` or `TFBertForQuestionAnswering`. These models are usually under `Question Answering` category and have `bert` in their labels
- Reference: [TFBertForQuestionAnswering](https://huggingface.co/transformers/model_doc/bert#transformers.TFBertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=bert&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [FardinSaboori/bert-finetuned-squad](https://huggingface.co/FardinSaboori/bert-finetuned-squad) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.
- In addition to the BERT model, we also need to save the `bertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import BertTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

MODEL_NAME = "FardinSaboori/bert-finetuned-squad"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

('onnx_models/FardinSaboori/bert-finetuned-squad/tokenizer_config.json',
 'onnx_models/FardinSaboori/bert-finetuned-squad/special_tokens_map.json',
 'onnx_models/FardinSaboori/bert-finetuned-squad/vocab.txt',
 'onnx_models/FardinSaboori/bert-finetuned-squad/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 421312
-rw-r--r-- 1 root root       690 Jun  9 20:14 config.json
-rw-r--r-- 1 root root 431187279 Jun  9 20:14 model.onnx
-rw-r--r-- 1 root root       695 Jun  9 20:14 special_tokens_map.json
-rw-r--r-- 1 root root      1273 Jun  9 20:14 tokenizer_config.json
-rw-r--r-- 1 root root    213450 Jun  9 20:14 vocab.txt


- Now we need to move `vocab.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find it.

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets

In [5]:
!ls -lR {EXPORT_PATH}

onnx_models/FardinSaboori/bert-finetuned-squad:
total 421104
drwxr-xr-x 2 root root      4096 Jun  9 20:14 assets
-rw-r--r-- 1 root root       690 Jun  9 20:14 config.json
-rw-r--r-- 1 root root 431187279 Jun  9 20:14 model.onnx
-rw-r--r-- 1 root root       695 Jun  9 20:14 special_tokens_map.json
-rw-r--r-- 1 root root      1273 Jun  9 20:14 tokenizer_config.json

onnx_models/FardinSaboori/bert-finetuned-squad/assets:
total 212
-rw-r--r-- 1 root root 213450 Jun  9 20:14 vocab.txt


Voila! We have our `vocab.txt` inside assets directory

## Import and Save BertForQuestionAnswering in Spark NLP


Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 24.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import BertForQuestionAnswering

spanClassifier = BertForQuestionAnswering.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your BertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 421156
-rw-r--r-- 1 root root 431253208 Jun  9 20:16 bert_classification_onnx
drwxr-xr-x 3 root root      4096 Jun  9 20:16 fields
drwxr-xr-x 2 root root      4096 Jun  9 20:16 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [12]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import BertForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = BertForQuestionAnswering.load(f"./{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([
    ["What's my name?", "My name is Clara and I live in Berkeley."]
]).toDF("question", "context")

result = pipeline.fit(example).transform(example)
result.select("answer.result").show(1, truncate=False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `BertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
